In [1]:
# DELETE
# Mount your google drive in google colabfrom google.colab import drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# DELETE
# Insert the directory
import sys
sys.path.insert(0,'/content/drive/MyDrive/simbsig/')
# Import worker classes
# from src.TestModule import TestClass
from simbsig.neighbors import RadiusNeighborsRegressor

# RadiusNeighborsRegressor Demo
This demo illustrates how SIMBSIG can be used for Nearest Neighbors queries, and how the use compares to scikit-learn. 

In [3]:
# from simbsig.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor as RadiusNeighborsRegressor_sk
import h5py as h5py
import numpy as np
import os

## Set Parameters

In [4]:
np.random.seed(98)
n_train_samples = 10
n_query_samples = 5
ylow = 0
yhigh = 10
n_dim = 2

## Create Toy Data
### numpy arrays

In [5]:
# numpy arrays
X_train = np.random.uniform(low=-5, high=5, size=(n_train_samples, n_dim))
X_query = np.random.uniform(low=-5, high=5, size=(n_query_samples, n_dim))
y_train = np.random.randint(low=ylow, high=yhigh, size=(n_train_samples))

### hdf5 files

In [6]:
# when working on colab, google drive can be used to save and read data
from google.colab import drive
drive.mount('/content/drive')

# depending on the structure of your google drive, you might want to choose a different dataset path
dataset_path = '/content/drive/MyDrive/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# hdf5 files using h5py
train_file = f'train.hdf5'
query_file = f'query.hdf5'

with h5py.File(os.path.join(dataset_path, f"{train_file}"), 'w') as f:
    f.create_dataset("X", data=X_train)
    f.create_dataset("y",data=y_train)

with h5py.File(os.path.join(dataset_path, f"{query_file}"), 'w') as f:
    f.create_dataset("X", data=X_query)

# open the hdf5 file for use
train_data = h5py.File(os.path.join(dataset_path, train_file), 'r')
query_data = h5py.File(os.path.join(dataset_path, query_file))

## Scikit-Learn


In [8]:
nn_sk = RadiusNeighborsRegressor_sk(radius=3)

nn_sk.fit(X_train, y_train)
nn_sk.predict(X_query)

array([ 6.00000000e+00,  8.00000000e+00, -9.22337204e+18,  5.00000000e+00,
        8.00000000e+00])

## SIMBSIG
### Using numpy arrays and CPU only
SIMBSIG can be used very similar to scikit-learn. In an existing workflow using scikit-learn, which may be on the verge of exceeding runtime or memory requirements, this allows a seamless transition to SIMBSIG.


In [9]:
nn = RadiusNeighborsRegressor(radius=3)

nn.fit(X_train, y_train)
nn.predict(X_query)

/content/drive/MyDrive/simbsig/simbsig/neighbors/RadiusNeighborsRegressor.py:170: UserWarning: The query points [2] have no neighbors within radius 3, leading to no meaningful prediction at these query points.
  warnings.warn(empty_query_point_neighborhood_msg)


array([ 6.00000000e+00,  8.00000000e+00, -9.22337204e+18,  5.00000000e+00,
        8.00000000e+00])

### Using hdf5 files and CPU only
If saving the entire data at once in the computer memory using numpy arrays is not reasonable anymore, the hdf5 file format can help. SIMBSIG can use data in hdf5 files, by setting the `mode` argument to `cpu`

In [12]:
nn_hdf5 = RadiusNeighborsRegressor(radius=3, mode='hdf5')

nn_hdf5.fit(train_data)
nn_hdf5.predict(query_data)

length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">
length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">
length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">
length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">


/content/drive/MyDrive/simbsig/simbsig/neighbors/RadiusNeighborsRegressor.py:170: UserWarning: The query points [2] have no neighbors within radius 3, leading to no meaningful prediction at these query points.
  warnings.warn(empty_query_point_neighborhood_msg)


10

### Using GPU acceleration
If data gets big, the execution time becomes an issue. SIMBSIG features GPU acceleration, by setting the `device` argument to `gpu`. This works with both inputs, numpy arrays and hdf5 files.

In [11]:
nn_hdf5 = RadiusNeighborsRegressor(radius=3, mode='hdf5', device='gpu')

nn_hdf5.fit(train_data)
nn_hdf5.predict(query_data)

length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 5
<HDF5 dataset "X": shape (5, 2), type "<f8">
length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">
length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">
length: 10
<HDF5 dataset "X": shape (10, 2), type "<f8">


/content/drive/MyDrive/simbsig/simbsig/neighbors/RadiusNeighborsRegressor.py:170: UserWarning: The query points [2] have no neighbors within radius 3, leading to no meaningful prediction at these query points.
  warnings.warn(empty_query_point_neighborhood_msg)


array([ 6.00000000e+00,  8.00000000e+00, -9.22337204e+18,  5.00000000e+00,
        8.00000000e+00])